# Data Pipeline

This is a data pipeline.

## 0 - Imports librarys

In [67]:
#1 - Generates DB with data that was used to screen whetehr participants met the inclusion/exclusion criteria to participate in the study.
##Imports libraries
import pandas as pd
import os
from pathlib import Path

## 1. Screening database
Generates DB with data that was used to screen whetehr participants met the inclusion/exclusion criteria to participate in the study.

### 1.1. Importing data

In [83]:
#Opens the excel file where data from participants are stored.
# Go up one folder (from /notebook to project root)
PROJ_ROOT = Path("..").resolve()

# Build the path safely
excel_part_data_path = PROJ_ROOT / "Screening_Data" / "data.xlsx"

# Read the Excel file
df_excel_data_part = pd.read_excel(excel_part_data_path)
##Drops redundant columns (empty columns regarding Q1,Q7,Q14,Q15) in the dataframe df_excel_data_part.
df_excel_data_part.drop(df_excel_data_part.columns[[33, *range(104,108)]], axis=1, inplace=True)

pd.set_option("display.max_columns", 106)  
df_excel_data_part

,participant,Idade_1,Sexo_1,Nacionalidade_1,FreqEnsSup_1,CicloEstudosEnsSup_1,CicloEstudosEnsSup_2,CicloEstudosEnsSup_3,CicloEstudosEnsSup_4,HabAcademicas_1,HabAcademicas_2,HabAcademicas_3,HabAcademicas_4,HabAcademicas_5,HabAcademicas_6,Curso_1,SituacaoLaboral_1,SituacaoLaboralTurnos_1,SituacaoLaboralTurnos_2,PresencaDoencasAnter_1,ListaDoencasAnter_1,MedicacaoPsicotropica_1,ListaMedicacaoPsicotropica_1,Fumador_1,NrCigarros_1,ProdutosSessaoTabagica_1,ProdutosSessaoTabagica_2,ConsomeCafeina_1,QuantidadeDiaCafeina_1,ConsomeAlcool_1,QuantidadeDiaAlcool_1,ConsomeDrogas_1,Viagens_1,Pergunta1_1,Pergunta2_1,Pergunta3_1,Pergunta4_1,Pergunta5_1,Pergunta6_1,Pergunta7_1,Pergunta8_1,Pergunta9_1,Pergunta10_1,Pergunta11_1,Pergunta12_1,Pergunta13_1,Pergunta14_1,Pergunta15_1,Pergunta16_1,MEQscore_1,BSI_1,BSI_2,BSI_3,BSI_4,BSI_5,BSI_6,BSI_7,BSI_8,BSI_9,BSI_10,BSI_11,BSI_12,BSI_13,BSI_14,BSI_15,BSI_16,BSI_17,BSI_18,BSI_19,BSI_20,BSI_21,BSI_22,BSI_23,BSI_24,BSI_25,BSI_26,BSI_27,BSI_28,BSI_29,BSI_30,BSI_31,BSI_32,BSI_33,BSI_34,BSI_35,BSI_36,BSI_37,BSI_38,BSI_39,BSI_40,BSI_41,BSI_42,BSI_43,BSI_44,BSI_45,BSI_46,BSI_47,BSI_48,BSI_49,BSI_50,BSI_51,BSI_52,BSI_53,TIME_start,TIME_end,TIME_total
0,9,18,2,1,1,1,0,0,0,0,0,0,0,0,0,Psicologia,2,0,0,2,NaN,2,NaN,3,NaN,0,0,1,1,2,NaN,2,2,example.2024-05-18-1850.data.dcaa517c-1e66-43c...,3,3,2,2,2,Pergunta7.2024-05-18-1851.data.dcaa517c-1e66-4...,4,0,1,1,2,3,Pergunta14.2024-05-18-1853.data.dcaa517c-1e66-...,Pergunta15.2024-05-18-1853.data.dcaa517c-1e66-...,2,37,3,0,0,1,2,2,0,0,1,2,1,0,1,2,1,2,3,1,0,2,3,1,0,2,1,0,2,0,1,0,1,1,0,1,2,3,1,1,1,2,3,1,0,1,0,1,0,1,0,2,2,2,3,2024-05-18-18-48,2024-05-18-18-57,9
1,11,18,2,1,1,1,0,0,0,0,0,0,0,0,0,psicologia,2,0,0,2,NaN,2,NaN,3,NaN,0,0,1,3,2,NaN,2,2,example.2024-07-19-1117.data.61704151-71d4-42c...,1,2,3,2,2,Pergunta7.2024-07-19-1118.data.61704151-71d4-4...,4,0,3,1,2,2,Pergunta14.2024-07-19-1119.data.61704151-71d4-...,Pergunta15.2024-07-19-1119.data.61704151-71d4-...,2,36,2,0,0,1,1,2,0,0,0,1,0,0,0,1,0,1,1,0,0,0,2,0,1,0,2,1,0,0,0,0,0,1,1,0,1,0,0,2,1,0,0,0,0,0,0,3,0,0,1,0,1,1,0,2024-07-19-11-16,2024-07-19-11-22,6
2,18,33,2,1,2,0,0,0,0,0,0,0,1,0,0,Medicina,1,0,1,1,Perturbação de ansiedade generalizada,1,Sertralina,3,NaN,0,0,1,1,2,NaN,2,2,example.2024-07-19-0832.data.b6b4e196-f5e7-419...,2,3,3,4,3,Pergunta7.2024-07-19-0832.data.b6b4e196-f5e7-4...,6,2,2,3,3,3,Pergunta14.2024-07-19-0834.data.b6b4e196-f5e7-...,Pergunta15.2024-07-19-0834.data.b6b4e196-f5e7-...,4,55,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,2,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2024-07-19-08-24,2024-07-19-08-37,13
3,20,31,1,1,1,0,0,1,0,0,0,0,0,0,0,Psicologia,1,0,1,2,NaN,2,NaN,3,NaN,0,0,1,1,2,NaN,2,2,example.2024-05-22-0022.data.e9a8934a-ddb9-46c...,3,3,3,2,3,Pergunta7.2024-05-22-0023.data.e9a8934a-ddb9-4...,2,2,2,2,2,3,Pergunta14.2024-05-22-0028.data.e9a8934a-ddb9-...,Pergunta15.2024-05-22-0029.data.e9a8934a-ddb9-...,2,38,2,0,0,0,1,1,0,2,0,0,0,0,0,1,1,1,1,0,2,0,1,0,0,0,1,2,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2024-05-22-00-19,2024-05-22-00-35,16


### 1.2. Replaces Quesitions 1,7,14, and 15 of the MEQ

In [85]:
df_excel_data_part.columns = [
    'participant', 'Idade', 'Sexo', 'Nacionalidade', 'FreqEnsSup',
    'CicloEstudosEnsSup_1','CicloEstudosEnsSup_2','CicloEstudosEnsSup_3','CicloEstudosEnsSup_4',
    'HabAcademicas_1','HabAcademicas_2','HabAcademicas_3','HabAcademicas_4','HabAcademicas_5','HabAcademicas_6',
    'Curso','SituacaoLaboral','SituacaoLaboralTurnos_1','SituacaoLaboralTurnos_2',
    'PresencaDoencasAnter','ListaDoencasAnter','MedicacaoPsicotropica','ListaMedicacaoPsicotropica',
    'Fumador','NrCigarros','ProdutosSessaoTabagica_1','ProdutosSessaoTabagica_2',
    'ConsomeCafeina','QuantidadeDiaCafeina','ConsomeAlcool','QuantidadeDiaAlcool','ConsomeDrogas','Viagens',
    'Pergunta1','Pergunta2','Pergunta3','Pergunta4','Pergunta5','Pergunta6','Pergunta7','Pergunta8',
    'Pergunta9','Pergunta10','Pergunta11','Pergunta12','Pergunta13','Pergunta14','Pergunta15','Pergunta16',
    'MEQscore',
    *[f'BSI:{i}' for i in range(1,54)],
    'TIME_start','TIME_end','TIME_total'
]

df_excel_data_part.sort_values(by='TIME_end', kind='mergesort', ascending=True, inplace=True)
df_excel_data_part.reset_index(drop=True, inplace=True)

# -------- 2) Build paths RELATIVE to the notebook (no hardcoded absolute paths)
PROJ_ROOT = Path("..").resolve()
meq_dir = PROJ_ROOT / "Screening_Data" / "experiment_data"

# -------- 3) Create a mapping: filename -> integer answer
# These .txt files are tiny (≈11B) and usually contain a single token/number.
file_to_value = {}
for fp in sorted(meq_dir.glob("*.txt")):
    try:
        # Plain-text read is most robust for these tiny files
        token = fp.read_text(encoding="utf-8", errors="ignore").strip().split()[0]
        file_to_value[fp.name] = int(token)
    except Exception as e:
        print(f"Skipping {fp.name}: {e}")

# Quick sanity:
# print(list(file_to_value.items())[:5])

# -------- 4) Vectorized replacement for Pergunta1/7/14/15
for col in ['Pergunta1','Pergunta7','Pergunta14','Pergunta15']:
    # Map filenames in the column to their numeric value; keep original if not found
    df_excel_data_part[col] = df_excel_data_part[col].map(file_to_value).fillna(df_excel_data_part[col])

# -------- 5) Display nicely
pd.set_option("display.max_columns", None)
df_excel_data_part.head(10)


,participant,Idade,Sexo,Nacionalidade,FreqEnsSup,CicloEstudosEnsSup_1,CicloEstudosEnsSup_2,CicloEstudosEnsSup_3,CicloEstudosEnsSup_4,HabAcademicas_1,HabAcademicas_2,HabAcademicas_3,HabAcademicas_4,HabAcademicas_5,HabAcademicas_6,Curso,SituacaoLaboral,SituacaoLaboralTurnos_1,SituacaoLaboralTurnos_2,PresencaDoencasAnter,ListaDoencasAnter,MedicacaoPsicotropica,ListaMedicacaoPsicotropica,Fumador,NrCigarros,ProdutosSessaoTabagica_1,ProdutosSessaoTabagica_2,ConsomeCafeina,QuantidadeDiaCafeina,ConsomeAlcool,QuantidadeDiaAlcool,ConsomeDrogas,Viagens,Pergunta1,Pergunta2,Pergunta3,Pergunta4,Pergunta5,Pergunta6,Pergunta7,Pergunta8,Pergunta9,Pergunta10,Pergunta11,Pergunta12,Pergunta13,Pergunta14,Pergunta15,Pergunta16,MEQscore,BSI:1,BSI:2,BSI:3,BSI:4,BSI:5,BSI:6,BSI:7,BSI:8,BSI:9,BSI:10,BSI:11,BSI:12,BSI:13,BSI:14,BSI:15,BSI:16,BSI:17,BSI:18,BSI:19,BSI:20,BSI:21,BSI:22,BSI:23,BSI:24,BSI:25,BSI:26,BSI:27,BSI:28,BSI:29,BSI:30,BSI:31,BSI:32,BSI:33,BSI:34,BSI:35,BSI:36,BSI:37,BSI:38,BSI:39,BSI:40,BSI:41,BSI:42,BSI:43,BSI:44,BSI:45,BSI:46,BSI:47,BSI:48,BSI:49,BSI:50,BSI:51,BSI:52,BSI:53,TIME_start,TIME_end,TIME_total
0,9,18,2,1,1,1,0,0,0,0,0,0,0,0,0,Psicologia,2,0,0,2,NaN,2,NaN,3,NaN,0,0,1,1,2,NaN,2,2,2,3,3,2,2,2,3,4,0,1,1,2,3,4,3,2,37,3,0,0,1,2,2,0,0,1,2,1,0,1,2,1,2,3,1,0,2,3,1,0,2,1,0,2,0,1,0,1,1,0,1,2,3,1,1,1,2,3,1,0,1,0,1,0,1,0,2,2,2,3,2024-05-18-18-48,2024-05-18-18-57,9
1,20,31,1,1,1,0,0,1,0,0,0,0,0,0,0,Psicologia,1,0,1,2,NaN,2,NaN,3,NaN,0,0,1,1,2,NaN,2,2,3,3,3,3,2,3,3,2,2,2,2,2,3,0,3,2,38,2,0,0,0,1,1,0,2,0,0,0,0,0,1,1,1,1,0,2,0,1,0,0,0,1,2,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2024-05-22-00-19,2024-05-22-00-35,16
2,18,33,2,1,2,0,0,0,0,0,0,0,1,0,0,Medicina,1,0,1,1,Perturbação de ansiedade generalizada,1,Sertralina,3,NaN,0,0,1,1,2,NaN,2,2,4,2,3,3,4,3,5,6,2,2,3,3,3,4,4,4,55,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,2,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2024-07-19-08-24,2024-07-19-08-37,13
3,11,18,2,1,1,1,0,0,0,0,0,0,0,0,0,psicologia,2,0,0,2,NaN,2,NaN,3,NaN,0,0,1,3,2,NaN,2,2,2,1,2,3,2,2,3,4,0,3,1,2,2,4,3,2,36,2,0,0,1,1,2,0,0,0,1,0,0,0,1,0,1,1,0,0,0,2,0,1,0,2,1,0,0,0,0,0,1,1,0,1,0,0,2,1,0,0,0,0,0,0,3,0,0,1,0,1,1,0,2024-07-19-11-16,2024-07-19-11-22,6
